Importar las librerias necesarias:


In [ ]:
# Herramientas de sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Modelos de sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

# Librerias complementarias
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Any

# Configurar visualización y desactivar warnings
%matplotlib inline
warnings.filterwarnings("ignore")

## 1. Recolección de la data


Obtener el dataset.

In [ ]:
def get_dataset(file_path: str) -> pd.DataFrame:
    """
    Función para obtener un dataset de una ruta específica.
    
    Parámetros:
        - file_path: La ruta al dataset.

    Devuelve:
    El dataset como un objeto DataFrame.
    """

    return pd.read_csv(file_path)

In [ ]:
df = get_dataset("raw_dataset.csv")

Estructura del dataset.

In [ ]:
df.head()

In [ ]:
df.info()

## 2. Preparación / preprocesamiento de la data


Herramientas para la estandarización de los datos.

In [ ]:
SCALER = StandardScaler()
LABEL_ENCODER = LabelEncoder()

Normalización del dataset.

In [ ]:
def get_normalized_dataset(dataset: pd.DataFrame) -> pd.DataFrame:
    """
    Normaliza un dataset proporcionado.

    Parámetros:
        - df: Dataset a normalizar.

    Devuelve: El dataset normalizado.
    """

    # a. Eliminación de características redundantes o innecesarias
    dataset.drop_duplicates(inplace=True)

    # b. Limpieza de filas nulas, vacías o con error
    dataset.replace(["", " ", "?", "None", "N/A", "na"], pd.NA, inplace=True)
    dataset = dataset.dropna()
    dataset.reset_index(drop=True, inplace=True)

    # c. Encoder o codificador a las características no numéricas
    dataset["smoker"].replace({"yes": 1, "no": 0}, inplace=True)
    dataset["sex"].replace({"male": 1, "female": 0}, inplace=True)
    dataset["region"] = LABEL_ENCODER.fit_transform(dataset["region"])

    # d. Normalizar y estandarizar la data con un escalador de datos
    num_data = dataset.select_dtypes(include="number")
    scaled_data = SCALER.fit_transform(num_data)
    dataset = pd.DataFrame(scaled_data, columns=num_data.columns)

    return dataset

In [ ]:
df = get_normalized_dataset(df)

## 3. Análisis descriptivo de la data (EDA)


#### a. Analisis de la data con gráficas

Conteo de Frecuencia para Variables Numericas

* Estas gráficas nos permiten visualizar la distribución de las diferentes variables numericas y detectar posibles sesgos o outliers.

In [ ]:
num_cols = df.select_dtypes(include=np.number).columns

for col in num_cols:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    sns.histplot(df[col].dropna(), kde=True, color='blue', ax=axes[0]) # type: ignore
    axes[0].set_title(f'Distribution of {col}')
    
    sns.boxplot(x=df[col], ax=axes[1])
    axes[1].set_title(f'Boxplot of {col}')
    
    plt.tight_layout()
    plt.show()

Conteo de Frecuencia para Variables Categóricas
* Usamos countplots para ver la distribución de frecuencias de las variables categóricas, como "sex", "smoker" y "region".

In [ ]:
cat_cols = df.select_dtypes(include='object').columns

if len(cat_cols) == 0:
    print("No categorical columns found in this dataset.")
else:
    for col in cat_cols:
        if df[col].nunique() < 20:
            plt.figure()
            sns.countplot(data=df, x=col, order=df[col].value_counts().index, palette='pastel')
            plt.xticks(rotation=45)
            plt.title(f'Distribution of {col}')
            plt.tight_layout()
            plt.show()

Relación entre "age" y "charges".
* Permite explorar la posible relación entre la edad del paciente y el monto de los gastos médicos.

In [ ]:
sns.scatterplot(x='age', y='charges', hue='smoker', data=df, palette='coolwarm', s=100)
plt.title('Relación entre Edad y Gastos Médicos')
plt.xlabel('Edad')
plt.ylabel('Gastos Médicos')
plt.legend(title='Fumador')
plt.show()

Relación entre "bmi" y "charges".

* En este diagrama se visualiza la relación entre el índice de masa corporal y los gastos médicos. 
* Se utiliza el color para diferenciar entre fumadores y no fumadores.

In [ ]:
sns.scatterplot(x='bmi', y='charges', hue='smoker', data=df, palette='Set1', s=100)
plt.title('Relación entre BMI y Gastos Médicos')
plt.xlabel('BMI')
plt.ylabel('Gastos Médicos')
plt.legend(title='Fumador')
plt.show()

Heatmap de Correlaciones

* Este mapa de calor muestra la correlación entre las variables numéricas y permite identificar relaciones potenciales.

In [ ]:
correlation = df[['age', 'bmi', 'children', 'charges']].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Mapa de Calor de Correlaciones')
plt.show()

#### b. Interpretación las estadísticas de los datos


Estadísticas del dataset.

In [ ]:
df.describe()

##### Observaciones

1. Edad (age)

La edad mínima es de 18 años y la máxima de 64, lo que indica que la muestra abarca adultos jóvenes hasta adultos de mediana edad o mayores dentro de este rango.

La media es aproximadamente 39.21 años, con una mediana cercana a 39 años, lo que sugiere que la distribución de las edades es simétrica o al menos no muy sesgada.

El primer cuartil (25%) es de 27 años, lo que significa que el 25% de la muestra tiene 27 años o menos.

El tercer cuartil (75%) es de 51 años, indicando que el 75% de los casos tienen menos de 51 años.

2. Índice de Masa Corporal (bmi)

El bmi varía desde un mínimo de 15.96 hasta un máximo de 53.13.

La media es aproximadamente 30.66, y la mediana es de 30.40. Esto señala que en promedio la población se encuentra en el rango de sobrepeso u obesidad, dado que un bmi superior a 30 se asocia generalmente a obesidad.

El 25% de los individuos tiene un bmi menor o igual a 26.30, un valor que se encuentra en el rango normal o ligeramente superior.

El 75% tiene un bmi menor o igual a 34.69, lo que implica que mientras muchos se encuentran en la zona del sobrepeso, una parte significativa presenta valores que indican obesidad.


3. Número de Hijos (children)

El mínimo indica que hay encuestados sin hijos y el máximo, que hay individuos con hasta 5 hijos.

La media es de aproximadamente 1.09 hijos, lo que sugiere que, en promedio, los encuestados tienen poco más de un hijo.

El percentil 25 es 0, lo que indica que el 25% de la muestra no tiene hijos. El percentil 75 es 2, lo que señala que el 75% de la población tiene hasta 2 hijos.

4. Cargos (charges)

La media de los cargos es de alrededor de 13270.42, mientras que la mediana es de 9382.03. La diferencia entre ambos (con una media mayor que la mediana) sugiere la presencia de valores atípicos o una distribución sesgada a la derecha.

25% de los cargos es igual o inferior a 4740.29, lo que indica que una parte significativa de la muestra incurre en costos relativamente bajos.

El 75% se sitúa en o por debajo de 16639.91, y los valores máximos muy altos empujan la media hacia arriba.

##### Conclusión general

La población encuestada está compuesta por adultos con edades comprendidas entre 18 y 64 años, con una distribución centrada en torno a los 39 años.

La mayoría de los individuos presenta valores de bmi en el rango de sobrepeso o incluso obesidad, lo que podría estar relacionado con ciertos riesgos para la salud. Además, la cantidad de hijos, predominantemente uno o dos, da una idea del contexto familiar típico en este grupo.

Los cargos presentan una variabilidad considerable, lo que puede estar asociado a factores como la edad, el estado de salud (reflejado en el bmi) y quizás el número de hijos, lo que puede influir en el costo de seguros o tratamientos médicos.


## 4. Entrenamiento del modelo


Modelos a entrenar.

In [ ]:
MODEL_NAMES = [
    "Ordinary Least Squares",
    "Ridge Regression",
    "Bayesian Regression",
    "Lasso Regression",
    "Nearest Neighbors Regression",
    "Random Forest Regression",
    "SVM Regression",
    "Neural Network MLP Regression",
]

##### Setup inicial

a. División del dataset en entradas y salidas/etiquetas (x, y).

In [ ]:
# Variables de entrada
X = df.drop("charges", axis=1)

# Variable de salida
y = df["charges"]

b. División del dataset en entrenamiento y testeo.

In [ ]:
# (80% entrenamiento, 20% testeo)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

##### Creación de los modelos

In [ ]:
# Ordinary Least Squares Regression
ols_model = LinearRegression()

# Ridge Regression
ridge_model = Ridge(alpha=1.0)

# Bayesian Regression
bayesian_model = BayesianRidge()

# Lasso Regression
lasso_model = Lasso(alpha=0.1)

# Nearest Neighbors Regression
knn_model = KNeighborsRegressor(n_neighbors=5)

# Random Forest Regression
rf_model = RandomForestRegressor(n_estimators=100, random_state=1, n_jobs=-1)

# SVM (Support Vector Machine) Regression
svm_model = SVR(kernel="rbf", C=1.0, epsilon=0.2)

# Neural Network MLP Regression
mlp_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=1)

In [ ]:
# Emparejar cada modelo con su nombre.
MODELS = {
    k: v
    for k, v in zip(
        MODEL_NAMES,
        [
            ols_model,
            ridge_model,
            bayesian_model,
            lasso_model,
            knn_model,
            rf_model,
            svm_model,
            mlp_model,
        ],
    )
}

Entrenamiento de los modelos.

In [ ]:
for model in MODELS.values():
    model.fit(X_train, y_train)

## 5. Validación y testeo del modelo


#### a. Análisis de performance


Función para obtener las métricas de cada modelo.

In [ ]:
def get_all_metrics(_X_test=X_test, _y_test=y_test, models: dict[str, Any] = MODELS) -> pd.DataFrame:
    """
    Calcula y devuelve las métricas de evaluación de
    todos los modelos entrenados.

    Parámetros:
    _X_test : pd.DataFrame, opcional
        Conjunto de datos de prueba con las variables
        predictoras. Por defecto usa "X_test".

    _y_test : pd.Series o np.array, opcional
        Conjunto de valores reales (etiquetas) correspondientes a
        las muestras de prueba. Por defecto usa "y_test".

    models :
        Los modelos a usar.

    Devuelve:
        Un dataframe que representa todas las métricas.
    """

    metrics = {
        model_name: {
            "R² Score": model.score(_X_test, _y_test),
            "MSE": mean_squared_error(_y_test, model.predict(_X_test)), # type: ignore
        }
        for model_name, model in models.items()
    }


    return pd.DataFrame(metrics).T

In [ ]:
METRICS = get_all_metrics()

##### - R² Score

In [ ]:
def show_R2(df: pd.DataFrame) -> None:
    """
    Muestra un gráfico de barras que compara el R² Score
    de cada modelo.

    Parámetros:
    df : pandas.DataFrame
        DataFrame que contiene, al menos, la columna "R² Score"
        y cuyos índices representan los nombres de los modelos.

    Retorna: None
        La función únicamente muestra el gráfico, sin retornar ningún valor.
    """

    plt.subplot(1, 2, 1)
    sns.barplot(x=df.index, y=df["R² Score"], palette="viridis")
    plt.title("Comparativa R² Score por Modelo")
    plt.xlabel("Modelo")
    plt.ylabel("R² Score")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

In [ ]:
show_R2(METRICS)

##### - MSE

In [ ]:
def show_MSE(df: pd.DataFrame) -> None:
    """
    Muestra un gráfico de barras que compara el MSE
    de cada modelo.

    Parámetros:
    df : pandas.DataFrame
        DataFrame que contiene al menos la columna "MSE" y
        cuyos índices representan los nombres de los modelos.

    Retorna: None
        La función únicamente muestra el gráfico, sin retornar
        ningún valor.
    """

    plt.subplot(1, 2, 2)
    sns.barplot(x=df.index, y=df["MSE"], palette="rocket_r")
    plt.title("Comparativa MSE por Modelo")
    plt.xlabel("Modelo")
    plt.ylabel("MSE")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

In [ ]:
show_MSE(METRICS)

#### b. Selección del algoritmo óptimo


##### Modelo seleccionado

El análisis de las métricas sugiere que el modelo de Neural Network MLP es el más adecuado para este problema.

In [ ]:
MODELO_OPTIMO = mlp_model

##### Justificación

1. El R² Score es de 0.847072, el cual es el más alto de todos los modelos evaluados, es decir, este modelo explica aproximadamente el 84.7% de la variabilidad de los datos.

2. El MSE (Error Cuadrático Medio) es de 0.175550, el cual es el más bajo de todos los modelos evaluados.

3. Una red neuronal (MLP) es capaz de aprender y entender relaciones complejas y no lineales con los predictores, como es el caso de los efectos combinados de edad, bmi y hábito de fumar.

## 6. Despliegue del modelo y comprobación con data recién creada


##### a. Conversión de data nueva cruda a formato de entrada del algoritmo

1. Ruta del nuevo dataset.

In [ ]:
dataset_nuevo = "test_dataset.csv"

2. Carga de datos y estandarización.

In [ ]:
new_df = get_dataset(dataset_nuevo)
new_df = get_normalized_dataset(new_df)

##### b. Predicción de categoría del dato

Datos de entrada y de salida del nuevo dataset.

In [ ]:
new_X_test = new_df.drop("charges", axis=1)
new_y_test = new_df["charges"]

Función para obtener las predicciones.

In [ ]:
def predict(X_test, models: dict[str, Any] = MODELS) -> pd.DataFrame:
    """
    Genera predicciones utilizando los modelos entrenados
    para un conjunto de datos de prueba.

    Parámetros:
        - X_test: El conjuto de datos de prueba de entrada.
        - models: Los modelos a usar para predecir.
    
    Retorna: pandas.DataFrame
        Un DataFrame que contiene las predicciones de cada modelo (limitado a las primeras filas).
    """

    predictions = {}
    for model_name, model in models.items():
        predictions[model_name] = model.predict(X_test)

    return pd.DataFrame(predictions).head()

In [ ]:
predict(new_X_test)

Mostrar el rendimiento de cada modelo.

In [ ]:
def show_performance(X_test, y_test, models: dict[str, Any] = MODELS) -> None:
    """
    Calcula y muestra las métricas de rendimiento de todos los
    modelos para un conjunto de datos de prueba.

    Parámetros:
        - X_test: El conjuto de datos de prueba de entrada.
        - y_test: El conjuto de datos de prueba de salida.
        - models: Los modelos a testear.

    Retorna: None
        Solo muestra las gráficas.
    """

    metrics = get_all_metrics(X_test, y_test, models)
    show_R2(metrics)
    show_MSE(metrics)

In [ ]:
show_performance(new_X_test, new_y_test)